# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails

#from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.39.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl_heart_failure_exp'

experiment=Experiment(ws, experiment_name)

## Dataset

In [3]:
default_ds = ws.get_default_datastore()

if 'Heart Failure' not in ws.datasets:
    default_ds.upload_files(files=['./heart_failure_clinical_records_dataset.csv'], # Upload the diabetes csv files in /data
                        target_path='data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='Heart Failure',
                                description='Clinic data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


In [4]:
#folder name created in hyperdrive
#import os, shutil

# Create a folder for the experiment files
folder_name = 'training'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

## Get data

Data has been registered using Hyperdrive, if not run hyperdrive first

In [5]:
# Split the dataset into training and validation subsets
# Get the training dataset
heart_ds = ws.datasets.get("Heart Failure")
#heart_ds=heart_ds.to_pandas_dataframe()


train_ds, test_ds = heart_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

Data ready!


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
#from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=training_cluster,
                             task = "classification",
                             training_data=train_ds,
                             validation_data = test_ds,
                             label_column_name="DEATH_EVENT",   
                             path = experiment_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings

)

In [8]:
from azureml.widgets import RunDetails

# TODO: Submit your experiment
print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'heart-automl-sdk')
remote_run = automl_experiment.submit(automl_config)

Submitting Auto ML experiment...
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart-automl-sdk,AutoML_7d88ea04-4d00-4249-89a6-42973cabd9a9,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
heart-automl-sdk,AutoML_7d88ea04-4d00-4249-89a6-42973cabd9a9,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

***********************************************************************

{'runId': 'AutoML_7d88ea04-4d00-4249-89a6-42973cabd9a9',
 'target': 'aml-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-04-03T12:05:08.849519Z',
 'endTimeUtc': '2022-04-03T12:43:48.633999Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'aml-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-automl-sdk","subscription_id":"6eee9ed4-ef02-4ad0-bbbd-194153ad2069","resource_group":"aml-resource","workspace_name":"aml-workspace","region":"westeurope","compute_target":"aml-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
'''
for run in automl_run.get_children():
    print('Run ID', run.id)
    for metric in run.get_metrics():
        print('\t', run.get_metrics(metric))
'''

In [11]:
# Get the best run, and its metrics and arguments
best_run,fitted_model= remote_run.get_output()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC_weighted'])
print(' -Accuracy:', best_run_metrics['accuracy'])
print(' -Arguments:',script_arguments)

Best Run Id:  AutoML_7d88ea04-4d00-4249-89a6-42973cabd9a9_47
 -AUC: 0.958168456755229
 -Accuracy: 0.8888888888888888
 -Arguments: []


In [12]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print('\nBest Model Definition:')
print(fitted_model)
print('\nBest Run Transformations:')
for step in fitted_model.named_steps:
    print(step)
print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: heart-automl-sdk,
Id: AutoML_7d88ea04-4d00-4249-89a6-42973cabd9a9_47,
Type: azureml.scriptrun,
Status: Completed)

Best Model Definition:
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/moun...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('34', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight='balanced', criterion='entropy', max_depth=None, max_features=0.3, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, m

In [13]:
import joblib
import os
from azureml.core import Model
#model = ws.models['AutoML_heart_model']
#print(model.name, 'version', model.version)

# Save the trained model
model_file = 'heart_model.pkl'
joblib.dump(value=Model, filename=model_file)
remote_run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)


In [28]:
'''
import joblib
import os

from azureml.core import Model
# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=Model, filename='outputs/AutoML_heart_model.pkl')
'''

['outputs/AutoML_heart_model.pkl']

In [18]:
#best_run.properties['model_name']

'AutoML6c8cea2b343'

In [14]:
#TODO: Save the best model

from azureml.core import Model

#model_path='outputs/AutoML_heart_model.pkl', properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']}
# Register model
#remote_run.register_model(model_name='AutoML_heart_model',
#                        tags={'Training context':'Auto ML'})


remote_run.register_model(model_name='AutoML_heart_model',
                        tags={'Training context':'AutoML'}
                        )


'''
best_run.register_model(model_path='./', model_name='AutoML_heart_model',
                        tags={'Training context':'AutoML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})
'''
print('Model registered')

Model registered


In [25]:
from azureml.core import Model

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

AutoML_heart_model version: 2
	 Training context : AutoML
	 AUC : 0.958168456755229
	 Accuracy : 0.8888888888888888




## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
model = ws.models['AutoML_heart_model']
print(model.name, 'version', model.version)

AutoML_heart_model version 1


In [33]:
#print(model.name)

AutoML_heart_model


Create a folder for configuration files

In [16]:
import os

folder_name = 'heart_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

print(folder_name, 'folder created.')

heart_service folder created.


In [56]:
%%writefile $folder_name/score_heart.py
import json
import joblib
import numpy as np
#import os
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    #model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'heart_model.pkl')
    model_path=Model.get_model_path('AutoML_heart_model')
    model = joblib.load(model_path)


def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = [1, 0]
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Overwriting heart_service/score_heart.py


In [62]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model


# Configure the scoring environment
inference_config = InferenceConfig(environment=best_run.get_environment(),  #runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="score_heart.py")
                                   #conda_file="heart_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 4)

service_name = "heart-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config,overwrite=True)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-04-03 17:26:11+00:00 Creating Container Registry if not exists.
2022-04-03 17:26:11+00:00 Registering the environment.
2022-04-03 17:26:11+00:00 Use the existing image.
2022-04-03 17:26:12+00:00 Generating deployment configuration.
2022-04-03 17:26:13+00:00 Submitting deployment to compute.
2022-04-03 17:26:16+00:00 Checking the status of deployment heart-service..
2022-04-03 17:28:28+00:00 Checking the status of inference endpoint heart-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [63]:
endpoint = service.scoring_uri
#print(endpoint)

TODO: In the cell below, send a request to the web service you deployed to test it.

Enable Application insights and get service logs

In [54]:
# enable application insight
service.update(enable_app_insights=True)

logs = service.get_logs()

for line in logs.split('\n'):
    print(line)
#print('Swagger URI: {}'.format(service.swagger_uri))

2022-04-03T16:16:15,675359000+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
2022-04-03T16:16:15,675789900+00:00 - nginx/run 
2022-04-03T16:16:15,675155400+00:00 - iot-server/run 
Starting HTTP server
2022-04-03T16:16:15,683582000+00:00 - rsyslog/run 
rsyslogd: /azureml-envs/azureml_2039c3f5cc4ae631ef3fdf7a2e093d5a/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-04-03T16:16:15,893483900+00:00 - iot-server/finish 1 0
2022-04-03T16:16:15,896834700+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (82)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 110
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2022-04-03 16:16:19,303 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found.

In [61]:
import requests
import urllib.request
import json


x_new=train_ds.to_pandas_dataframe().iloc[0:3,0:12]
# Convert the array to a serializable list in a JSON document
#input_json = json.dumps({"data": x_new})
input_json=str.encode(json.dumps({'data': x_new.to_dict(orient='records')}))

# Set the content type
headers = { 'Content-Type':'application/json' }

req = urllib.request.Request(endpoint, input_json, headers = headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


b'"{\\"error\\": \\"DataException:\\\\n\\\\tMessage: The number of features in [fitted data](12) does not match with those in [input data](1). Please inspect your data, and make sure that features are aligned in both the Datasets.\\\\n\\\\tInnerException: None\\\\n\\\\tErrorResponse \\\\n{\\\\n    \\\\\\"error\\\\\\": {\\\\n        \\\\\\"code\\\\\\": \\\\\\"UserError\\\\\\",\\\\n        \\\\\\"message\\\\\\": \\\\\\"The number of features in [fitted data](12) does not match with those in [input data](1). Please inspect your data, and make sure that features are aligned in both the Datasets.\\\\\\",\\\\n        \\\\\\"target\\\\\\": \\\\\\"X\\\\\\",\\\\n        \\\\\\"inner_error\\\\\\": {\\\\n            \\\\\\"code\\\\\\": \\\\\\"BadData\\\\\\",\\\\n            \\\\\\"inner_error\\\\\\": {\\\\n                \\\\\\"code\\\\\\": \\\\\\"InvalidDimension\\\\\\",\\\\n                \\\\\\"inner_error\\\\\\": {\\\\n                    \\\\\\"code\\\\\\": \\\\\\"DataShapeMismatch\\\\\\"\

'\npredicted_classes = json.loads(predictions.json())\n\nfor i in range(len(x_new)):\n    print ("Patient {}".format(x_new[i]), predicted_classes[i] )\n'

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
